# Cutout Size Determination by detector

Cutout size varies with max dy, which varies by detector. This notebook can read a conf file and determine the max dy. Specifically, the scipy.optimize function minimize returns the minimum -dy (negative dy) and it's location (the negative of the minimum -dy is the maximum). The absolute value of this is the minimum size cutout.

1) Put the conf file in.
2) Run the notebook.
3) Note the value returned at 'x:'; this is the location of the maximum.
4) Note the value returned at 'func:'; this times -1 is the maximum value.
5) Test somewhere that setting size equal to this func value works, or one greater. This test is in case scipy got stuck in a local max/min.
6) Track the detector value.

det1: 77  - tested \
det2: 80 \
det3: 88 \
det4: 180 \
det5: 195 \
det6: 209 \
det7: 279 \
det8: 302 \
det9: 328 \
det10: 1 - use compute_size (object based sizing)? \
det11: 2 \
det12: 6 \
det13: 30 \
det14: 18 \
det15: 2 \
det16: 78 \
det17: 49 \
det18: 22

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import minimize, Bounds

import os
os.chdir("/Users/keith/astr/research_astr/roman_configs")

In [2]:
from collections import OrderedDict
config_file = "Roman.det1.07242020.conf"

conf = OrderedDict()
lines = open(config_file).readlines()

for line in lines:
    if (line.startswith('#')) | (line.strip() == '') | ('"' in line):
                continue

    if line.startswith("DYDX_A_"):
        spl = line.split(';')[0].split('#')[0].split()
        param = spl[0]
        value = np.cast[float](spl[1:])
        conf[param] = value

b = {}
for n, dydx in enumerate(conf.keys()):
    for m, coef in enumerate(conf[dydx]):
        b[f"{n},{m}"] = coef

In [3]:
a_0 = lambda i, j: b["0,0"] + b["0,1"]*i + b["0,2"]*j + b["0,3"]*i**2 + b["0,4"]*i*j + b["0,5"]*j**2
a_1 = lambda i, j: b["1,0"] + b["1,1"]*i + b["1,2"]*j + b["1,3"]*i**2 + b["1,4"]*i*j + b["1,5"]*j**2
a_2 = lambda i, j: b["2,0"] + b["2,1"]*i + b["2,2"]*j + b["2,3"]*i**2 + b["2,4"]*i*j + b["2,5"]*j**2


dy = lambda args: -(a_0(args[0],args[1]) + a_1(args[0],args[1])*args[2] + a_2(args[0],args[1])*args[2]**2)

In [4]:
bounds = Bounds([0,0,-800],[4800,4800,800])
minimize(dy, x0=[4800,4800,800], bounds=bounds)

  message: CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL
  success: True
   status: 0
      fun: -76.00948368
        x: [ 0.000e+00  0.000e+00 -8.000e+02]
      nit: 5
      jac: [ 5.051e-03  1.451e-02  1.134e-02]
     nfev: 36
     njev: 9
 hess_inv: <3x3 LbfgsInvHessProduct with dtype=float64>